In [3]:
import requests
# address = "시 구 동 번지"
def address_to_xy(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    key = "카카오 rest api key" # 카카오 rest api key
    headers = {"Authorization": f"KakaoAK {key}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    result = response.json()["documents"][0]["address"]
    
    return result["x"], result["y"]


In [4]:
# 카카오맵 API를 이용하여 주소를 위도와 경도로 변환
kakao_api_key = "카카오 rest api key"

def get_lat_lng(addr):
    url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(addr)
    headers = {"Authorization": "KakaoAK {}".format(kakao_api_key)}
    result = requests.get(url, headers=headers)
    json_obj = result.json()
    for document in json_obj['documents']:
        return document['y'], document['x'] # 위도, 경도 반환

In [5]:
from math import radians, cos, sin, asin, sqrt

# haversine 공식을 이용한 두 위도, 경도 사이의 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [148]:
import pandas as pd
df_reservation = pd.read_csv('reservation.csv', encoding='CP949')

# 46row 전처리
df_reservation = df_reservation[df_reservation['destAddress'].str.split(' ').str[0].isin(['서울', '서울시', '서울특별시'])]

df_reservation.to_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

In [149]:
# 지번 이후 상세주소 제거
import re

df_reservation = pd.read_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

for i in range(len(df_reservation)):
    address = df_reservation['destAddress'][i]
    try:
        number_index = re.search(r'\d+\b', address).end()
        df_reservation.at[i, 'destAddress'] = address[:number_index]
    except AttributeError:
        df_reservation.drop(i, inplace=True)

df_reservation.to_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

In [150]:
df_reservation = pd.read_csv('df_reservation_hospital.csv', encoding='utf-8-sig')
df_reservation['dest_x'] = 0
df_reservation['dest_y'] = 0
df_reservation.to_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

In [156]:
df_reservation.to_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

In [155]:
# 목적지 경도, 위도

df_reservation = pd.read_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

for i in range(len(df_reservation)):
    address = df_reservation['deptAddress'][i]
    try:
        x, y = address_to_xy(address)
        df_reservation['dest_x'][i], df_reservation['dest_y'][i] = x, y
    except IndexError:
        df_reservation.drop(i, inplace=True)
    

C:\Users\admin\AppData\Local\Temp\ipykernel_3796\2532614420.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reservation['dest_x'][i], df_reservation['dest_y'][i] = x, y
C:\Users\admin\AppData\Local\Temp\ipykernel_3796\2532614420.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reservation['dest_x'][i], df_reservation['dest_y'][i] = x, y
C:\Users\admin\AppData\Local\Temp\ipykernel_3796\2532614420.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [6]:
import requests
import json
import pandas as pd
import io

KAKAO_API_KEY = '카카오 rest api key'  
SEARCH_PLACE_API_URL = 'https://dapi.kakao.com/v2/local/search/keyword.json'
query = '서울 동물병원'

headers = {"Authorization": "KakaoAK " + KAKAO_API_KEY}
seoul_hospitals = []

for page in range(1, 46):
    params = {
        "query": query,
        "page": page,
        "size": 15
    }
    res = requests.get(SEARCH_PLACE_API_URL, headers=headers, params=params)
    res_json = json.loads(res.text)

    for document in res_json['documents']:
        seoul_hospitals.append(document)


In [7]:
import pandas as pd

# pandas 데이터프레임으로 변환
seoul_hospitals_df = pd.DataFrame(seoul_hospitals)


In [159]:
seoul_hospitals_df.to_csv('df_seoul_hospitals.csv', encoding='utf-8-sig')

In [16]:
km = [0.2, 0.5, 1, 2, 3]
for k in km:
    df_reservation = pd.read_csv('df_reservation_hospital.csv', encoding='utf-8-sig')
    df_reservation['is_near_hospital'] = 0 
    df_reservation.to_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

    df_reservation = pd.read_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

    for i in range(len(df_reservation)):
        dest_lat, dest_lon = df_reservation['dest_y'][i], df_reservation['dest_x'][i]
        for h in seoul_hospitals:
            h_lat = float(h['y'])
            h_lon = float(h['x'])
            if haversine(dest_lat, dest_lon, h_lat, h_lon) <= k:  # 거리가 200m 이내인지 확인
                
                df_reservation['is_near_hospital'][i] = 1
                break

    df_reservation.to_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

    # near_hospital 컬럼의 값이 True인 행의 개수를 구합니다.
    near_hospital_count = len(df_reservation[df_reservation['is_near_hospital'] == 1])

    # 전체 행의 개수를 구합니다.
    total_count = len(df_reservation)

    # 통계치를 계산하고 출력합니다.
    print(f"전체 주소 중에서 동물병원 {k}km 이내 주소의 개수: ", near_hospital_count)
    print("전체 주소의 개수: ", total_count)
    print("동물병원 근처인 주소의 비율: ", near_hospital_count / total_count * 100, "%\n")

C:\Users\admin\AppData\Local\Temp\ipykernel_37364\387470673.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reservation['is_near_hospital'][i] = 1


전체 주소 중에서 동물병원 0.2km 이내 주소의 개수:  507
전체 주소의 개수:  4645
동물병원 근처인 주소의 비율:  10.914962325080731 %



C:\Users\admin\AppData\Local\Temp\ipykernel_37364\387470673.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reservation['is_near_hospital'][i] = 1


전체 주소 중에서 동물병원 0.5km 이내 주소의 개수:  1101
전체 주소의 개수:  4645
동물병원 근처인 주소의 비율:  23.70290635091496 %



C:\Users\admin\AppData\Local\Temp\ipykernel_37364\387470673.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reservation['is_near_hospital'][i] = 1


전체 주소 중에서 동물병원 1km 이내 주소의 개수:  2648
전체 주소의 개수:  4645
동물병원 근처인 주소의 비율:  57.0075349838536 %



C:\Users\admin\AppData\Local\Temp\ipykernel_37364\387470673.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reservation['is_near_hospital'][i] = 1


전체 주소 중에서 동물병원 2km 이내 주소의 개수:  3604
전체 주소의 개수:  4645
동물병원 근처인 주소의 비율:  77.58880516684607 %



C:\Users\admin\AppData\Local\Temp\ipykernel_37364\387470673.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reservation['is_near_hospital'][i] = 1


전체 주소 중에서 동물병원 3km 이내 주소의 개수:  4080
전체 주소의 개수:  4645
동물병원 근처인 주소의 비율:  87.83638320775027 %



In [12]:
df_reservation = pd.read_csv('df_reservation_hospital.csv', encoding='utf-8-sig')
df_reservation['is_near_hospital'] = 0 
df_reservation.to_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

In [13]:
df_reservation = pd.read_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

for i in range(len(df_reservation)):
    dest_lat, dest_lon = df_reservation['dest_y'][i], df_reservation['dest_x'][i]
    for h in seoul_hospitals:
        h_lat = float(h['y'])
        h_lon = float(h['x'])
        if haversine(dest_lat, dest_lon, h_lat, h_lon) <= 2:  # 거리가 200m 이내인지 확인
            df_reservation['is_near_hospital'][i] = 1
            break

df_reservation.to_csv('df_reservation_hospital.csv', encoding='utf-8-sig')

0
2
3
4
5
6
8
9
10
13
14
15
16
18
19
20
21
24
25
26
27
29
30
31
32
35
36
37
39
40
42
44
45
47
49
51
52
53
54
56
58
59
60
61
62
63
64
66
67
68
69
70
71
72
73
74
76
79
80
83
84
85
86
87
88
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
109
110
111
112
113
114
115
116
117
118
122
123
124
125
126
128
130
131
133
135
137
139
140
141
144
146
148
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
167
169
172
173
175
177
179
180
181
182
183
184
186
187
189
190
191
192
193
194
195
196
198
199
201
202
203
204
205
207
209
210
211
212
213
214
216
217
218
219
220
221
222
224
225
226
227
228
229
230
232
233
234
235
236
237
238
239
240
243
244
246
247
251
254
255
256
257
258
259
260
261
262
263


C:\Users\admin\AppData\Local\Temp\ipykernel_37364\911491282.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reservation['is_near_hospital'][i] = 1


265
266
267
268
269
270
272
273
274
278
279
281
284
285
287
288
289
291
292
293
294
295
296
297
298
301
302
303
304
305
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
330
331
335
336
337
338
344
350
351
352
353
354
355
356
357
358
359
360
361
365
367
368
369
370
371
372
373
374
375
376
377
378
379
383
384
385
387
388
389
390
393
394
396
399
400
401
402
404
407
409
410
411
412
413
414
415
416
417
418
419
422
423
424
426
428
429
430
432
434
435
436
437
438
439
440
441
442
443
444
447
448
449
450
451
452
453
454
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
494
495
496
497
498
499
502
504
505
507
508
509
510
511
514
515
516
517
518
519
520
521
522
523
524
526
527
528
529
530
531
532
533
534
535
536
537
538
539
544
545
547
548
549
550
551
553
555
556
557
558
561
562
563
564
565
566
567
568
569
570
572
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589


In [14]:
# near_hospital 컬럼의 값이 True인 행의 개수를 구합니다.
near_hospital_count = len(df_reservation[df_reservation['is_near_hospital'] == 1])

# 전체 행의 개수를 구합니다.
total_count = len(df_reservation)

# 통계치를 계산하고 출력합니다.
print("전체 주소 중에서 동물병원 근처인 주소의 개수: ", near_hospital_count)
print("전체 주소의 개수: ", total_count)
print("동물병원 근처인 주소의 비율: ", near_hospital_count / total_count * 100, "%")

전체 주소 중에서 동물병원 근처인 주소의 개수:  3604
전체 주소의 개수:  4645
동물병원 근처인 주소의 비율:  77.58880516684607 %
